In [222]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.neural_network import MLPRegressor  # veya MLPRegressor
from sklearn.metrics import classification_report, mean_squared_error, r2_score  # görevi neyse
import joblib  # model kaydetmek için
import matplotlib as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [ ]:
# Define the model - Bu hücreyi her çalıştırdığınızda model sıfırdan oluşur
def create_model():
    model = Sequential([
        Input(shape=(57,)),
        Dense(units=32, activation='relu'),
        Dropout(0.3),
        Dense(units=32, activation='relu'),
        Dropout(0.3),
        BatchNormalization(),
        Dense(units=128, activation='relu'),
        Dropout(0.4),
        BatchNormalization(),
        Dense(units=64, activation='relu'),
        Dropout(0.2),
        Dense(units=32, activation='relu'),
        Dropout(0.2),
        Dense(units=32, activation='relu'),
        Dense(1)
    ])
    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=["mae"])
    return model

In [224]:
# Early Stopping - daha sabırlı ayarlar
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=15,  # 25 epoch iyileşme yoksa dur
    min_delta=0.01,  # 0.001'den küçük iyileşmeleri sayma
    restore_best_weights=True,
    verbose=1
)

# Learning Rate Scheduler - öğrenmeyi iyileştirir
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,  # Learning rate'i yarıya indir
    patience=10,  # 10 epoch iyileşme yoksa LR'yi düşür
    min_lr=1e-6,
    verbose=1
)

model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

In [225]:
data = pd.read_csv('scaled_data.csv')
data['popularity'] = np.log(data['popularity'] + 1)
#data = data[:10000]
X = data.drop('popularity', axis=1)
y = data['popularity']

In [226]:
# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Eğitim seti boyutu: {X_train.shape}")
print(f"Test seti boyutu: {X_test.shape}")
print(f"Feature sayısı: {X_train.shape[1]}")

Eğitim seti boyutu: (22684, 57)
Test seti boyutu: (5672, 57)
Feature sayısı: 57


In [227]:
# Model'i TEK seferde düzgün eğit
# Yeni model oluştur (önceki ağırlıkları temizler)
model = create_model()

history = model.fit(
    X_train, y_train, 
    validation_split=0.2, 
    epochs=150,  # Daha fazla epoch, early stopping zaten durduracak
    callbacks=[early_stopping, reduce_lr, model_checkpoint], 
    batch_size=64,
    verbose=1
)

print("\n✅ Eğitim tamamlandı!")
print(f"📊 En iyi model 'best_model.keras' dosyasına kaydedildi.")

# En iyi validation loss değerini bul
best_val_loss = min(history.history['val_loss'])
best_epoch = history.history['val_loss'].index(best_val_loss) + 1

print(f"\n🎯 En İyi Validation Loss: {best_val_loss:.5f}")
print(f"📍 En İyi Epoch: {best_epoch}")

Epoch 1/150
266/284 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.2560
Epoch 1: val_loss improved from None to 2.53544, saving model to best_model.keras
284/284 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 2.6963 - val_loss: 2.5354 - learning_rate: 0.0010
Epoch 2/150
284/284 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8973
Epoch 2: val_loss improved from 2.53544 to 2.25574, saving model to best_model.keras
284/284 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1.8174 - val_loss: 2.2557 - learning_rate: 0.0010
Epoch 3/150
275/284 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.6834
Epoch 3: val_loss improved from 2.25574 to 1.92908, saving model to best_model.keras
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 1.6294 - val_loss: 1.9291 - learning_rate: 0.0010
Epoch 4/150
278/284 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.4714
Epoch 4: val_loss improved from 1.92908 to 1.83175, saving model to best_model.keras
284/284 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 1.4497 - val_loss: 1.8318 - learning_rate: 

In [229]:
# Test seti üzerinde değerlendirme
test_loss = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss (Log-transformed): {test_loss:.4f}")

# Tahminleri al
y_pred_log = model.predict(X_test, verbose=0)

# Log'dan orijinal değerlere dönüştür
y_pred_original = np.expm1(y_pred_log).flatten()
y_test_original = np.expm1(y_test)

# Orijinal ölçekteki performans metrikleri
from sklearn.metrics import mean_absolute_error
mae_original = mean_absolute_error(y_test_original, y_pred_original)
mse_original = mean_squared_error(y_test_original, y_pred_original)
rmse_original = np.sqrt(mse_original)
r2_original = r2_score(y_test_original, y_pred_original)

print(f"\n--- Orijinal Ölçekte Metrikler ---")
print(f"MAE: {mae_original:.2f}")
print(f"RMSE: {rmse_original:.2f}")
print(f"MSE: {mse_original:.2f}")
print(f"R² Score: {r2_original:.4f}")

178/178 - 0s - 2ms/step - loss: 0.8285
Test Loss (Log-transformed): 0.8285

--- Orijinal Ölçekte Metrikler ---
MAE: 16.26
RMSE: 20.41
MSE: 416.53
R² Score: 0.2614
